In [29]:
using Plots
using Random
using Distributions
using LinearAlgebra
include("pogm_restart_original.jl") # issues adding MIRT so will use julia file instead
rng = Random.seed!(0)
N = 100
D = 100
d = 10
σ1 = 2
σ2 = 35
goodpts = 10
X = zeros(D,N)
Y = zeros(D,N)
Π = vec(zeros(N,1))
U1 = svd(rand(D,N)).U[:,1:d]
for k = 1:N
    X[:,k] = U1*U1'*rand(Uniform(-100,100),D)
end
for j=1:N
        if (j<= goodpts)
            Y[:,j] = X[:,j] + rand(Normal(0,σ1),D)
            Π[j] = σ1^2
        else
            Y[:,j] = X[:,j] + rand(Normal(0,σ2),D)
            Π[j] = σ2^2
        end
end
function HPCA_POGM(Y, λr, w, N, rank, ϵ)
    Π = w.^-1
    Lf = minimum(w)
    Π = Diagonal(Π)
    #x0 = zeros(size(Y))
    U_svd = svd(Y).U[:,1:N]
    x0 = deepcopy(U_svd*U_svd'*Y)
    grad = K -> -1*(Y-K)*Π
    soft = (x,t) -> sign.(x) .* max.(abs.(x) .- t, 0)
    function pssvt(x,t,N)
        U,S,V = svd(x)
        S[(N+1):end] = soft.(S[(N+1):end],t)
        return U*diagm(S)*V'
    end
    prox1 = (z,c) -> pssvt(z, c*λr, N)
    K, _ = pogm_restart(x0, x -> 0, grad, Lf ; g_prox=prox1, niter=ϵ) 
    U = svd(K).U[:,1:rank]
    return K
end
function HPCA_ADMM(Y, λr, w, N, μ, ρ, d, U_init)
    #X = zeros(size(Y))
    #Z = zeros(size(Y))
    X = deepcopy(U_init*U_init'*Y)
    Z = deepcopy(Y-X)
    Π = Diagonal(w.^-1)
    #Λ = zeros(size(Y))
    Λ2 = sign.(Y)
    Λ = deepcopy(Λ2 ./ (max(opnorm(Λ2), (1/λr)*norm(Λ2, Inf))))
    normY = norm(Y,2)
    count = 0
    soft = (x,t) -> sign.(x) .* max.(abs.(x) .- t, 0)
    function pssvt(x,t,N)
        U,S,V = svd(x)
        S[(N+1):end] = soft.(S[(N+1):end],t)
        return U*diagm(S)*V'
    end
    #while norm(Y-X-Z,2)/normY > 1e-6
    #while norm(X-X0)/norm(X0) > 1e-6
    for i = 1:2000
        #X0 = X
        X = pssvt(Y-Z+(1/μ)*Λ, λr/μ,N)
        Z = μ*(Y-X+(1/μ)*Λ)*inv(Π+μ*I)
        Λ = Λ + μ*(Y-X-Z)
        μ = ρ*μ
        count = count + 1
    end
    U = svd(X).U[:,1:d]   
    return U
end
function weightedPCA(Y , w, k)
    L = unique(w)
    Σ = zeros(size(Y)[1], size(Y)[1])
    for i=1:length(L)
        ind = findall(x -> x == L[i], w)
        Σ = Σ + L[i]*(Y[:,ind]*Y[:,ind]')
    end
    U = reverse(eigvecs(Σ), dims=2)
    Ȳ = reshape(shuffle(Y[:]), size(Y))
    U_Y = svd(Y).S
    U_Ȳ = svd(Ȳ).S
    #k = floor(Int, norm(U_Y .> U_Ȳ, 0))
    return U[:,1:k] #, k
end
U_WPCA = weightedPCA(Y,Π.^-1,d)
error_wpca_10 = norm(U_WPCA*U_WPCA' - U1*U1', 2)/norm(U1*U1', 2)
U_WPCA = weightedPCA(Y,Π.^-1,8)
error_wpca_8 = norm(U_WPCA*U_WPCA' - U1*U1', 2)/norm(U1*U1', 2)
U_WPCA = weightedPCA(Y,Π.^-1,12)
error_wpca_12 = norm(U_WPCA*U_WPCA' - U1*U1', 2)/norm(U1*U1', 2)

0.5871530004726269

In [13]:
λr = vcat(0:0.1:1,1:0.5:10)
error_hpca_zero = zeros(size(λr))
λr[1] = 0.001
for i=1:length(λr)
    println(i)
    flush(stdout)
    U_HPCA = HPCA_ADMM(Y, λr[i], Π, 10, 0.01, 1.005, d, zeros(100,10)) 
    error_hpca_zero[i] = norm(U_HPCA*U_HPCA' - U1*U1',2)/norm(U1*U1',2)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [21]:
λr = vcat(0:0.1:1,1:0.5:10)
error_hpca_rank8 = zeros(size(λr))
λr[1] = 0.001
for i=1:length(λr)
    println(i)
    flush(stdout)
    U_HPCA = HPCA_ADMM(Y, λr[i], Π, 8, 0.01, 1.005, 8, zeros(100,10)) 
    error_hpca_rank8[i] = norm(U_HPCA*U_HPCA' - U1*U1',2)/norm(U1*U1',2)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [31]:
λr = vcat(0:0.1:1,1:0.5:10)
error_hpca_rank12 = zeros(size(λr))
λr[1] = 0.001
for i=1:length(λr)
    println(i)
    flush(stdout)
    U_HPCA = HPCA_ADMM(Y, λr[i], Π, 12, 0.01, 1.005, 12, zeros(100,10)) 
    error_hpca_rank12[i] = norm(U_HPCA*U_HPCA' - U1*U1',2)/norm(U1*U1',2)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [37]:
U_svd = svd(Y).U[:,1:10]
error_svd = norm(U_svd*U_svd' - U1*U1', 2)/norm(U1*U1', 2)

0.8902024700565202

In [45]:
using LaTeXStrings
#plot(λr, error_rpca, label="HPCA (α=0)", title="HPCA Rank Knowledge (zero init.)", xlabel="λr", ylabel=L"\Vert U_{*} U_{*}^{'}- UU' \Vert_F/ \Vert UU' \Vert_F",linewidth=2)
#plot(λr, error_hpca_rank8, label="HPCA (α=8)",linewidth=2, c=:seagreen, linestyle=:dash, legend_title_font_pointsize=18)
plot(λr, error_hpca_zero, label="HPCA (α=10)",linewidth=2,title="HPCA Rank Knowledge (ν known, zero init.)", xlabel="λr", ylabel=L"\Vert U_{*} U_{*}^{'}- UU' \Vert_F/ \Vert UU' \Vert_F",c=:deepskyblue, legendfontsize=14.0)
#plot(λr, error_hpca_zero, label="HPCA (α=10)",linewidth=2,title="HPCA Rank Knowledge (νo init.)", xlabel="λr", ylabel=L"\Vert U_{*} U_{*}^{'}- UU' \Vert_F/ \Vert UU' \Vert_F",c=:deepskyblue)
plot!(λr, error_hpca_rank8, label="HPCA (α=8)",linewidth=2, c=:seagreen)
plot!(λr, error_hpca_rank12, label="HPCA (α=8)",linewidth=2, c=:orange)
hline!([error_wpca_8], label="", linewidth=2, c=:seagreen,linestyle=:dash)
hline!([error_wpca_10], label="", c=:deepskyblue, linewidth=2, linestyle=:dash)
hline!([error_wpca_12], label="", linewidth=2, c=:orange,linestyle=:dash)
annotate!(8, error_wpca_10+0.025, "WPCA r=10", :deepskyblue)
annotate!(8, error_wpca_8+0.025, "WPCA r=8", :seagreen)
annotate!(8, error_wpca_12+0.025, "WPCA r=12", :orange)
annotate!(3, error_svd-0.025, "PPCA r=10", :purple2)
hline!([error_svd], label="", linewidth=2, c=:purple2)
#hline!([error_heppcat], label="HePPCAT",linewidth=2)
savefig("HPCA_KNOWN_RANK_KNOWLEDGE_ZERO_INIT.pdf")

In [46]:
heppCAT

LoadError: UndefVarError: heppCAT not defined